In [141]:
from advent2019.utils import *
from assertpy import assert_that
from itertools import combinations
from collections import defaultdict
from collections import deque
from decimal import *
import math

map = read_strings("day10.data")

map_1 = read_strings("day10_1.data")
map_2 = read_strings("day10_2.data")
map_3 = read_strings("day10_3.data")
map_4 = read_strings("day10_4.data")
map_5 = read_strings("day10_5.data")

In [140]:
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

def fit_line(p1, p2):
    (x1, y1), (x2, y2) = sorted((p1, p2))
    if x2 == x1:
        return Decimal('-Infinity'), Decimal(x1)        
    else:    
        a = Decimal((y2 - y1) / (x2 - x1))        
        b = Decimal(y1 - a * x1)
        b = round(b, 4)        
        return round(a, 4), b

def get_asteroid_lines(asteroid_map):
    asteroids = {}
    for y, row in enumerate(asteroid_map):
        for x, c in enumerate(row):
            if c != '.':
                asteroids[(x, y)] = 0
                
    lines = defaultdict(set)
    for a1, a2 in combinations(asteroids.keys(), 2):
        a, b = fit_line(a1, a2)
        lines[(a, b)] |= { a1, a2 } 
    
    for k, ax_set in lines.items():        
        ax = sorted(ax_set)                
        asteroids[ax[0]] += 1
        for a in ax[1:-1]:
            asteroids[a] += 2            
        asteroids[ax[-1]] += 1
        
    return asteroids, lines
        
def asteroids_from_best_location(asteroid_map):
    
    asteroids, lines = get_asteroid_lines(asteroid_map)            
    return max(asteroids.items(), key=lambda kv: kv[1])

assert_that(asteroids_from_best_location(map_1)).is_equal_to(((3,4), 8))
assert_that(asteroids_from_best_location(map_3)).is_equal_to(((1,2), 35))
assert_that(asteroids_from_best_location(map_4)).is_equal_to(((6,3), 41))
assert_that(asteroids_from_best_location(map_2)).is_equal_to(((5,8), 33))
assert_that(asteroids_from_best_location(map_5)).is_equal_to(((11,13), 210))

In [139]:
%%time
assert_that(asteroids_from_best_location(map)).is_equal_to(((13,17), 269)) 

Wall time: 843 ms


In [155]:
def find_Nth_to_vaporize(n, rays: list):    
    queue = deque(rays)
    i = 0
    cur_a = None
    while i < n:
        cur_ray = queue.popleft()        
        cur_a = cur_ray.pop(0)
        if cur_ray:
            queue.append(cur_ray)
        i += 1
        
    return cur_a

def line_to_rays(lines, station):
    rays = []
    
    for (a, b), ax_set in lines.items():
        if station in ax_set:
            ax = sorted(ax_set)
            
            if a == Decimal("-infinity"):
                ax.reverse()
            
            i = ax.index(station)
            before = ax[0:i]
            after = ax[i+1:]            
            
            if a < 0:       # negative slope
                if before:
                    rays.append(((3, a), before))
                if after:
                    rays.append(((1, a), after))
            else:           # positive slope
                if before:
                    rays.append(((4, a), before))
                if after:
                    rays.append(((2, a), after))
            
    rays = [r for k, r in sorted(rays, key=lambda kr: kr[0])]
    
    # for r in rays:
    #     print(r)
        
    return rays

def find_Nth(asteroid_map, n):
    
    asteroids, lines = get_asteroid_lines(asteroid_map)
    station = max(asteroids.items(), key=lambda kv: kv[1])[0]
    
    print(f"Station: {station}")
    rays = line_to_rays(lines, station)
    
    return find_Nth_to_vaporize(n, rays)

assert_that(find_Nth(map_5, 200)).is_equal_to((8,2))

Station: (11, 13)


In [158]:
%%time
assert_that(find_Nth(map, 200)).is_equal_to((6,12))

Station: (13, 17)
Wall time: 1.06 s
